### Optimizing polynomials with strange ways

Author: Raido Everest

In [ ]:
# Scipy minimize appears to work locally - depending on the starting point, we may arrive
# at a different end position. This might bias end results toward methods that initialize
# in many different places, but I highly doubt it makes much of a difference here.

import random
import numpy as np
from scipy.optimize import minimize  # checking goodness of result compared to scipy

In [ ]:
# Function to generate functions to test on
def quartic():
    # default ranges for variables too
    a, b = random.random() * 4.9 + 0.1, random.random() * 10 - 5
    c, d = random.random() * 40 - 20, random.random() * 500 - 250
    
    return lambda x: a * x**4 + b * x**3 + c * x**2 + d * x

# Making an nd-quartic function to test higher dimensionalities.
# For the sake of generality, this should be used for the single variable case as well.
# n - how many inputs the function takes
# outputs the sum of n random quartic functions
def quartic_n(n):
    fs = [quartic() for _ in range(n)]
    return lambda answers: sum(map(lambda pair: pair[0](pair[1]), zip(fs, answers)))

# Example scipy optimization of a 4d quartic function:
# minimize(quartic_n(4), (0,0,0,0))

### Differential Evolution

In [ ]:
# The main function that returns the best set of inputs it finds.
# f - the function.
# n - input dimensionality
# k - population size
# scaling - the scaling parameter when creating a new input set
# loops   - how many loops it will do before giving up finding a better solution
# Outputs the minimum value of the function that it found and the necessary input for it
def diff_evo(f, n, k = 80, scaling = 0.5, loops=25):
    # Create initial input set
    pop       = create_population(n, k)
    fitnesses = calculate_fitness(f, pop)
    best, bestval = getbest(pop, fitnesses)   # pair of best input and its value
    
    loops_since_improvement = 0  # Keep it going until it's not working anymore.
    while loops_since_improvement < loops:
        loops_since_improvement += 1
        
        # Create next population by mutating previous one
        newpop       = create_next_population(pop, scaling)
        newfitnesses = calculate_fitness(f, newpop)
        nextbest, nextbestval = getbest(newpop, newfitnesses)
        
        # Keep track of what the best outcome is
        if nextbestval < bestval:
            best, bestval = nextbest, nextbestval
            loops_since_improvement = 0
        
        # Always choose the better one of the two choices to represent the 'next generation'
        for i in range(k):
            if newfitnesses[i] < fitnesses[i]:  # if something must be changed
                pop[i], fitnesses[i] = newpop[i], newfitnesses[i]
    
    # Return the best value and its inputs
    return bestval, best

# Creates a population not knowing any previous information
# n - dimensionality of output
# k - population size
def create_population(n, k):
    # Arbitrary range, but should be fine for the time being.
    return [[random.random() * 200 - 100 for _ in range(n)] for _ in range(k)]

# Creates the next generation of input sets
# pop - the old population
# scaling - the scaling parameter
def create_next_population(pop, scaling = 0.5):
    dim = len(pop[0])
    n = len(pop)
    
    newpop = [None] * n
    for i in range(n):
        a, b = random.randint(0, n-1), random.randint(0, n-1)  # Indices of two random elements
        diff = [(pop[a][d] - pop[b][d]) for d in range(dim)]   # Difference of two random input vectors
        newpop[i] = [pop[i][d] + diff[d] * scaling for d in range(dim)]  # Mutated input has been created
    
    return newpop

# Just makes a list of evaluation results
def calculate_fitness(f, pop):
    return [f(inputs) for inputs in pop]

# Given a population and fitnesses, returns the best element and its fitness.
def getbest(pop, fitnesses):
    best, bestfitness = pop[0], fitnesses[0]
    for i in range(1, len(fitnesses)):
        if fitnesses[i] < bestfitness:
            best, bestfitness = pop[i], fitnesses[i]
    return best, bestfitness

### Particle Swarm Optimization

In [ ]:
# f - the function.
# n - input dimensionality
# k - population size
# scaling - the scaling parameter when creating a new input set
# loops   - how many loops it will do before giving up finding a better solution
# Outputs the minimum value of the function that it found and the necessary input for it
def pso(f, n, k=25, loops=25):
    # Create initial population - including velocity, personal best locations
    pop, velocity, pb_locs = create_pso(n, k)
    # Also calculate personal best actual values.
    pb_vals = calculate_fitness(f, pb_locs)
    vals    = pb_vals[::]
    g_best_loc, g_best_val = getbest(pop, vals)
    
    loops_since_improvement = 0
    while loops_since_improvement < loops:
        loops_since_improvement += 1
        # Create the next generation - updates population, velocity
        iterate_pso(pop, velocity, pb_locs, g_best_loc)
        # Now update values, personal bests, global bests
        vals = calculate_fitness(f, pop)
        update_personal_best(pb_vals, pb_locs, vals, pop)
        next_best_loc, next_best_val = getbest(pop, vals)
        if next_best_val < g_best_val:
            loops_since_improvement = 0
            g_best_loc, g_best_val = next_best_loc, next_best_val
        
    return g_best_val, g_best_loc  # best output and input

# n - input dimensionality
# k - population size
def create_pso(n, k):
    pop = create_population(n, k)  # Just use the same population init as DE
    velocity = norm(create_population(n, k), 10)
    pb_locs  = pop[::]
    return pop, velocity, pb_locs

# Iterates the PSO state.
# pop - current locations
# velocity - how fast we are moving and where
# pb_locs - the best positions value wise each element has been to
# best_loc - globally the best position that everybody also wants to move toward
def iterate_pso(pop, velocity, pb_locs, best_loc, c1=0.5, c2=0.5):
    for i in range(len(pop)):
        z1, z2 = random.random(), random.random()
        velocity[i] = list(np.add(velocity[i],
                                  np.add(c1*z1*np.subtract(pb_locs[i], pop[i]), c2*z2*np.subtract(best_loc, pop[i]))))
        pop[i] = list(np.add(pop[i], velocity[i]))
    
    if random.random() < 0.1:
        norm(velocity, 10)  # I will basically reset the speed my swarm moves at randomly - seems to help...
    
# does what it says
def update_personal_best(pb_vals, pb_locs, vals, locs):
    for i in range(len(vals)):
        if vals[i] < pb_vals[i]:
            pb_vals[i], pb_locs[i] = vals[i], locs[i]
    
# updates a list of vectors in place such that they become unit vectors
# vectors - list of vectors that all have the same dimensions
def norm(vectors, tolength=1):
    if len(vectors) == 0:
        return vectors
    dim = len(vectors[0])
    # length of vector is the sqrt of its dot product with itself
    # simply divide each component with that value
    for i in range(len(vectors)):
        length = np.dot(vectors[i], vectors[i])**0.5
        for j in range(dim):
            vectors[i][j] /= length * tolength
    return vectors

### Simulated Annealing

### Genetic Algorithm

### Testing Goodness of Differential Evolution

I expect scipy and diff_evo to be pretty close overall by quality of result. Turns out, they are, at least in this test.

In [ ]:
# Generate a bunch of functions. Use scipy and diff_evo to find optimal solutions.
# Arbitrarily choose amount of functions for each dimensionality.
functions = 10

print('Measuring difference of scipy and diff_evo - the higher, the better for DE')
for dimensions in range(1,5):
    print(f'Testing out {dimensions} dimensions...')
    for _ in range(functions):
        function = quartic_n(dimensions)

        sp_ans = minimize(function, [0] * dimensions)
        sp_bestval, sp_bestinput = sp_ans.fun, sp_ans.x

        de_bestval, de_bestinput = diff_evo(function, dimensions)

        print(f'Difference between scipy and diffevo: {sp_bestval - de_bestval}')
    print()

### Testing Goodness of Particle Swarm Optimization

It's not bad. Sometimes gets a really good result in the end, thousands below the scipy default optimizer.

In [ ]:
# Generate a bunch of functions. Use scipy and diff_evo to find optimal solutions.
# Arbitrarily choose amount of functions for each dimensionality.
functions = 10

print('Measuring difference of scipy and PSO - the higher, the better for PSO')
for dimensions in range(1,5):
    print(f'Testing out {dimensions} dimensions...')
    for _ in range(functions):
        function = quartic_n(dimensions)

        sp_ans = minimize(function, [0] * dimensions)
        sp_bestval, sp_bestinput = sp_ans.fun, sp_ans.x

        pso_bestval, pso_bestinput = pso(function, dimensions)

        print(f'Difference between scipy and PSO: {sp_bestval - pso_bestval}')
    print()

### Testing Goodness of Simulated Annealing

### Testing Goodness of Genetic Algorithm